In [1]:
import json
import os
import pandas as pd
import datetime
import httplib, urllib
from itertools import izip_longest
from datetime import timedelta, time, date
import datetime
import csv
from matplotlib import pyplot as plt

def grouper(iterable, n, fillvalue=None):
    args = [iter(iterable)] * n
    return izip_longest(*args, fillvalue=fillvalue)

In [44]:
# Setting some constants
use_lines = ['OR', 'SV', 'BL']
input_dir = '../data/'
departure = "L'Enfant Plaza"
arrival = "Foggy Bottom-GWU"
min_time = '15:00:00'
max_time = '17:30:00'
time_relevant = lambda x: min_time < x < max_time
is_on_time = lambda x: time(16, 50) < x < time(17,0)
outcomes_file = '../data/lenfant_foggybottom_class.csv'
bin_seconds = 180

In [3]:
# Getting routes data
routes_infile_name = os.path.join(input_dir, 'routes.json')
with open(routes_infile_name, 'r') as routes_infile:
    routes_data = json.loads(routes_infile.read().strip())['StandardRoutes']
print routes_data
track_circuits_byline = [x['TrackCircuits'] for x in routes_data]
all_track_circuits = [item for sublist in track_circuits_byline for item in sublist]
circuits_station_codes = {circuit['CircuitId']: circuit['StationCode'] for circuit in all_track_circuits if circuit['StationCode'] is not None}

[{u'TrackCircuits': [{u'StationCode': None, u'SeqNum': 0, u'CircuitId': 2603}, {u'StationCode': u'J03', u'SeqNum': 1, u'CircuitId': 2604}, {u'StationCode': None, u'SeqNum': 2, u'CircuitId': 2605}, {u'StationCode': None, u'SeqNum': 3, u'CircuitId': 2606}, {u'StationCode': None, u'SeqNum': 4, u'CircuitId': 2607}, {u'StationCode': None, u'SeqNum': 5, u'CircuitId': 2608}, {u'StationCode': None, u'SeqNum': 6, u'CircuitId': 2609}, {u'StationCode': None, u'SeqNum': 7, u'CircuitId': 2610}, {u'StationCode': None, u'SeqNum': 8, u'CircuitId': 2611}, {u'StationCode': None, u'SeqNum': 9, u'CircuitId': 2612}, {u'StationCode': None, u'SeqNum': 10, u'CircuitId': 2613}, {u'StationCode': None, u'SeqNum': 11, u'CircuitId': 2614}, {u'StationCode': None, u'SeqNum': 12, u'CircuitId': 2615}, {u'StationCode': None, u'SeqNum': 13, u'CircuitId': 2616}, {u'StationCode': None, u'SeqNum': 14, u'CircuitId': 2617}, {u'StationCode': None, u'SeqNum': 15, u'CircuitId': 2618}, {u'StationCode': None, u'SeqNum': 16, u'Cir

In [4]:
# Getting stations data
stations_infile_name = os.path.join(input_dir, 'stations.json')
with open(stations_infile_name, 'r') as stations_infile:
    stations_data = json.loads(stations_infile.read().strip())
stations_by_line = [stations_data[x]['Stations'] for x in stations_data]
all_stations = [item for sublist in stations_by_line for item in sublist]
codes_stations = {x['Code']: x['Name'] for x in all_stations}
circuits_stations = {circuit_id: codes_stations[station_code] for circuit_id, station_code in circuits_station_codes.items()}
circuits_stations

{7: u'Shady Grove',
 32: u'Rockville',
 53: u'Twinbrook',
 62: u'White Flint',
 80: u'Grosvenor-Strathmore',
 95: u'Medical Center',
 109: u'Bethesda',
 126: u'Friendship Heights',
 133: u'Tenleytown-AU',
 142: u'Van Ness-UDC',
 154: u'Cleveland Park',
 164: u'Woodley Park-Zoo/Adams Morgan',
 179: u'Dupont Circle',
 190: u'Farragut North',
 203: u'Metro Center',
 210: u'Shady Grove',
 232: u'Rockville',
 251: u'Twinbrook',
 260: u'White Flint',
 278: u'Grosvenor-Strathmore',
 294: u'Medical Center',
 309: u'Bethesda',
 326: u'Friendship Heights',
 336: u'Tenleytown-AU',
 346: u'Van Ness-UDC',
 356: u'Cleveland Park',
 363: u'Woodley Park-Zoo/Adams Morgan',
 378: u'Dupont Circle',
 389: u'Farragut North',
 467: u'Gallery Pl-Chinatown',
 477: u'Judiciary Square',
 485: u'Union Station',
 496: u'NoMa-Gallaudet U',
 513: u'Rhode Island Ave-Brentwood',
 527: u'Brookland-CUA',
 548: u'Fort Totten',
 571: u'Takoma',
 591: u'Silver Spring',
 611: u'Forest Glen',
 629: u'Wheaton',
 652: u'Glenm

In [5]:
# Combining routes and stations data to get mapping of track positions to stations
track_circuits_byline = [x['TrackCircuits'] for x in routes_data]
all_track_circuits = [item for sublist in track_circuits_byline for item in sublist]
circuits_station_codes = {circuit['CircuitId']: circuit['StationCode'] for circuit in all_track_circuits if circuit['StationCode'] is not None}
circuits_stations = {circuit_id: codes_stations[station_code] for circuit_id, station_code in circuits_station_codes.items()}
circuits_stations

{7: u'A15',
 32: u'A14',
 53: u'A13',
 62: u'A12',
 80: u'A11',
 95: u'A10',
 109: u'A09',
 126: u'A08',
 133: u'A07',
 142: u'A06',
 154: u'A05',
 164: u'A04',
 179: u'A03',
 190: u'A02',
 203: u'A01',
 210: u'A15',
 232: u'A14',
 251: u'A13',
 260: u'A12',
 278: u'A11',
 294: u'A10',
 309: u'A09',
 326: u'A08',
 336: u'A07',
 346: u'A06',
 356: u'A05',
 363: u'A04',
 378: u'A03',
 389: u'A02',
 467: u'B01',
 477: u'B02',
 485: u'B03',
 496: u'B35',
 513: u'B04',
 527: u'B05',
 548: u'B06',
 571: u'B07',
 591: u'B08',
 611: u'B09',
 629: u'B10',
 652: u'B11',
 661: u'A01',
 667: u'B01',
 677: u'B02',
 686: u'B03',
 700: u'B35',
 717: u'B04',
 731: u'B05',
 757: u'B06',
 785: u'B07',
 809: u'B08',
 828: u'B09',
 846: u'B10',
 868: u'B11',
 944: u'C15',
 955: u'C14',
 969: u'C13',
 976: u'C12',
 1010: u'C10',
 1024: u'C09',
 1036: u'C08',
 1052: u'C07',
 1070: u'C06',
 1092: u'C05',
 1105: u'C04',
 1117: u'C03',
 1126: u'C02',
 1135: u'C01',
 1137: u'C15',
 1148: u'C14',
 1162: u'C13',


In [43]:
# Iterate through input files and get all into one big data frame, extracting relevant data from .jsons

out_df = pd.DataFrame(columns=['DateTime', 'TrainID', 'Line', 'Direction','StationName'])

for infile_names_chunk in grouper(glob.glob(os.path.join(input_dir, '2018*')), 1000):
    out_rows = []
    for infile_name in infile_names_chunk:
        if infile_name:
            dt = datetime.datetime.strptime(os.path.basename(infile_name), '%Y-%m-%d_%H:%M:%S')
            print dt
            try:
                with open(infile_name, 'r') as infile:
                    train_positions = [train for train in json.loads(infile.read().strip())['TrainPositions'] if train['ServiceType'] == 'Normal']
                    times_trains_stations = [(str(dt), train['TrainId'], train['LineCode'], train['DirectionNum'], circuits_stations[train['CircuitId']]) for train in train_positions if train['CircuitId'] in circuits_stations]
                    out_rows.extend(times_trains_stations)
            except Exception as exp:
                print "Error: {0}".format(str(exp))
    out_df = pd.concat([out_df,pd.DataFrame(out_rows, columns=['DateTime', 'TrainID', 'Line', 'Direction','StationName'])], axis=0)
print len(out_df)

2018-06-10 13:59:00
2018-06-10 13:59:10
2018-06-10 13:59:20
2018-06-10 13:59:30
2018-06-10 13:59:40
2018-06-10 13:59:50
2018-06-10 14:00:01
2018-06-10 14:00:11
2018-06-10 14:00:21
2018-06-10 14:00:31
2018-06-10 14:00:41
2018-06-10 14:00:51
2018-06-10 14:01:02
2018-06-10 14:01:12
2018-06-10 14:01:22
2018-06-10 14:01:32
2018-06-10 14:01:42
2018-06-10 14:01:52
2018-06-10 14:02:02
2018-06-10 14:02:13
2018-06-10 14:02:23
2018-06-10 14:02:33
2018-06-10 14:02:43
2018-06-10 14:02:54
2018-06-10 14:03:04
2018-06-10 14:03:15
2018-06-10 14:03:25
2018-06-10 14:03:35
2018-06-10 14:03:45
2018-06-10 14:03:55
2018-06-10 14:04:05
2018-06-10 14:04:16
2018-06-10 14:04:26
2018-06-10 14:04:36
2018-06-10 14:04:46
2018-06-10 14:04:56
2018-06-10 14:05:06
2018-06-10 14:05:17
2018-06-10 14:05:27
2018-06-10 14:05:37
2018-06-10 14:05:47
2018-06-10 14:05:57
2018-06-10 14:06:07
2018-06-10 14:06:18
2018-06-10 14:06:28
2018-06-10 14:06:38
2018-06-10 14:06:48
2018-06-10 14:06:58
2018-06-10 14:07:08
2018-06-10 14:07:19


2018-06-10 15:35:56
2018-06-10 15:36:06
2018-06-10 15:36:16
2018-06-10 15:36:26
2018-06-10 15:36:36
2018-06-10 15:36:46
2018-06-10 15:36:56
2018-06-10 15:37:06
2018-06-10 15:37:17
2018-06-10 15:37:27
2018-06-10 15:37:37
2018-06-10 15:37:47
2018-06-10 15:37:58
2018-06-10 15:38:08
2018-06-10 15:38:18
2018-06-10 15:38:28
2018-06-10 15:38:38
2018-06-10 15:38:49
2018-06-10 15:38:59
2018-06-10 15:39:09
2018-06-10 15:39:19
2018-06-10 15:39:29
2018-06-10 15:39:39
2018-06-10 15:39:50
2018-06-10 15:40:00
2018-06-10 15:40:10
2018-06-10 15:40:20
2018-06-10 15:40:30
2018-06-10 15:40:40
2018-06-10 15:40:50
2018-06-10 15:41:00
2018-06-10 15:41:11
2018-06-10 15:41:21
2018-06-10 15:41:31
2018-06-10 15:41:41
2018-06-10 15:41:51
2018-06-10 15:42:02
2018-06-10 15:42:12
2018-06-10 15:42:22
2018-06-10 15:42:32
2018-06-10 15:42:42
2018-06-10 15:42:53
2018-06-10 15:43:03
2018-06-10 15:43:13
2018-06-10 15:43:23
2018-06-10 15:43:34
2018-06-10 15:43:44
2018-06-10 15:43:54
2018-06-10 15:44:04
2018-06-10 15:44:15


2018-06-10 17:08:51
2018-06-10 17:09:01
2018-06-10 17:09:11
2018-06-10 17:09:21
2018-06-10 17:09:31
2018-06-10 17:09:41
2018-06-10 17:09:51
2018-06-10 17:10:01
2018-06-10 17:10:11
2018-06-10 17:10:22
2018-06-10 17:10:32
2018-06-10 17:10:42
2018-06-10 17:10:52
2018-06-10 17:11:02
2018-06-10 17:11:12
2018-06-10 17:11:22
2018-06-10 17:11:33
2018-06-10 17:11:43
2018-06-10 17:11:53
2018-06-10 17:12:03
2018-06-10 17:12:13
2018-06-10 17:12:23
2018-06-10 17:12:33
2018-06-10 17:12:43
2018-06-10 17:12:54
2018-06-10 17:13:04
2018-06-10 17:13:14
2018-06-10 17:13:24
2018-06-10 17:13:34
2018-06-10 17:13:44
2018-06-10 17:13:55
2018-06-10 17:14:05
2018-06-10 17:14:15
2018-06-10 17:14:25
2018-06-10 17:14:35
2018-06-10 17:14:45
2018-06-10 17:14:56
2018-06-10 17:15:06
2018-06-10 17:15:16
2018-06-10 17:15:26
2018-06-10 17:15:36
2018-06-10 17:15:47
2018-06-10 17:15:57
2018-06-10 17:16:07
2018-06-10 17:16:17
2018-06-10 17:16:27
2018-06-10 17:16:37
2018-06-10 17:16:47
2018-06-10 17:16:57
2018-06-10 17:17:08


2018-06-10 18:46:54
2018-06-10 18:47:05
2018-06-10 18:47:15
2018-06-10 18:47:25
2018-06-10 18:47:35
2018-06-10 18:47:45
2018-06-10 18:47:55
2018-06-10 18:48:06
2018-06-10 18:48:16
2018-06-10 18:48:26
2018-06-10 18:48:36
2018-06-10 18:48:46
2018-06-10 18:48:56
2018-06-10 18:49:06
2018-06-10 18:49:17
2018-06-10 18:49:27
2018-06-10 18:49:37
2018-06-10 18:49:47
2018-06-10 18:49:57
2018-06-10 18:50:07
2018-06-10 18:50:18
2018-06-10 18:50:28
2018-06-10 18:50:38
2018-06-10 18:50:48
2018-06-10 18:50:58
2018-06-10 18:51:08
2018-06-10 18:51:18
2018-06-10 18:51:29
2018-06-10 18:51:39
2018-06-10 18:51:49
2018-06-10 18:51:59
2018-06-10 18:52:09
2018-06-10 18:52:19
2018-06-10 18:52:30
2018-06-10 18:52:40
2018-06-10 18:52:50
2018-06-10 18:53:00
2018-06-10 18:53:10
2018-06-10 18:53:20
2018-06-10 18:53:30
2018-06-10 18:53:40
2018-06-10 18:53:50
2018-06-10 18:54:01
2018-06-10 18:54:11
2018-06-10 18:54:21
2018-06-10 18:54:31
2018-06-10 18:54:41
2018-06-10 18:54:51
2018-06-10 18:55:02
2018-06-10 18:55:12


2018-06-10 20:12:51
2018-06-10 20:13:01
2018-06-10 20:13:11
2018-06-10 20:13:21
2018-06-10 20:13:32
2018-06-10 20:13:42
2018-06-10 20:13:52
2018-06-10 20:14:02
2018-06-10 20:14:12
2018-06-10 20:14:22
2018-06-10 20:14:33
2018-06-10 20:14:43
2018-06-10 20:14:53
2018-06-10 20:15:03
2018-06-10 20:15:13
2018-06-10 20:15:24
2018-06-10 20:15:34
2018-06-10 20:15:44
2018-06-10 20:15:54
2018-06-10 20:16:04
2018-06-10 20:16:14
2018-06-10 20:16:24
2018-06-10 20:16:34
2018-06-10 20:16:45
2018-06-10 20:16:55
2018-06-10 20:17:05
2018-06-10 20:17:15
2018-06-10 20:17:25
2018-06-10 20:17:35
2018-06-10 20:17:46
2018-06-10 20:17:56
2018-06-10 20:18:06
2018-06-10 20:18:16
2018-06-10 20:18:26
2018-06-10 20:18:36
2018-06-10 20:18:47
2018-06-10 20:18:57
2018-06-10 20:19:07
2018-06-10 20:19:17
2018-06-10 20:19:27
2018-06-10 20:19:37
2018-06-10 20:19:48
2018-06-10 20:19:58
2018-06-10 20:20:08
2018-06-10 20:20:18
2018-06-10 20:20:28
2018-06-10 20:20:38
2018-06-10 20:20:48
2018-06-10 20:20:58
2018-06-10 20:21:09


...

2018-06-23 08:29:46
2018-06-23 08:29:56
2018-06-23 08:30:06
2018-06-23 08:30:16
2018-06-23 08:30:26
2018-06-23 08:30:36
2018-06-23 08:30:46
2018-06-23 08:30:56
2018-06-23 08:31:07
2018-06-23 08:31:17
2018-06-23 08:31:27
2018-06-23 08:31:37
2018-06-23 08:31:47
2018-06-23 08:31:57
2018-06-23 08:32:07
2018-06-23 08:32:17
2018-06-23 08:32:27
2018-06-23 08:32:37
2018-06-23 08:32:48
2018-06-23 08:32:58
2018-06-23 08:33:08
2018-06-23 08:33:18
2018-06-23 08:33:28
2018-06-23 08:33:38
2018-06-23 08:33:48
2018-06-23 08:33:58
2018-06-23 08:34:08
2018-06-23 08:34:19
2018-06-23 08:34:29
2018-06-23 08:34:39
2018-06-23 08:34:49
2018-06-23 08:34:59
2018-06-23 08:35:09
2018-06-23 08:35:19
2018-06-23 08:35:30
2018-06-23 08:35:40
2018-06-23 08:35:50
2018-06-23 08:36:00
2018-06-23 08:36:10
2018-06-23 08:36:21
2018-06-23 08:36:31
2018-06-23 08:36:41
2018-06-23 08:36:51
2018-06-23 08:37:01
2018-06-23 08:37:11
2018-06-23 08:37:21
2018-06-23 08:37:31
2018-06-23 08:37:41
2018-06-23 08:37:51
2018-06-23 08:38:02


2018-06-23 09:43:39
2018-06-23 09:43:49
2018-06-23 09:43:59
2018-06-23 09:44:10
2018-06-23 09:44:20
2018-06-23 09:44:30
2018-06-23 09:44:40
2018-06-23 09:44:50
2018-06-23 09:45:00
2018-06-23 09:45:10
2018-06-23 09:45:20
2018-06-23 09:45:30
2018-06-23 09:45:40
2018-06-23 09:45:50
2018-06-23 09:46:01
2018-06-23 09:46:11
2018-06-23 09:46:21
2018-06-23 09:46:31
2018-06-23 09:46:41
2018-06-23 09:46:52
2018-06-23 09:47:02
2018-06-23 09:47:12
2018-06-23 09:47:22
2018-06-23 09:47:32
2018-06-23 09:47:42
2018-06-23 09:47:52
2018-06-23 09:48:02
2018-06-23 09:48:13
2018-06-23 09:48:23
2018-06-23 09:48:33
2018-06-23 09:48:43
2018-06-23 09:48:53
2018-06-23 09:49:03
2018-06-23 09:49:13
2018-06-23 09:49:23
2018-06-23 09:49:33
2018-06-23 09:49:44
2018-06-23 09:49:54
2018-06-23 09:50:04
2018-06-23 09:50:14
2018-06-23 09:50:24
2018-06-23 09:50:34
2018-06-23 09:50:44
2018-06-23 09:50:54
2018-06-23 09:51:04
2018-06-23 09:51:15
2018-06-23 09:51:25
2018-06-23 09:51:35
2018-06-23 09:51:45
2018-06-23 09:51:55


2018-06-23 11:02:16
2018-06-23 11:02:26
2018-06-23 11:02:36
2018-06-23 11:02:47
2018-06-23 11:02:57
2018-06-23 11:03:07
2018-06-23 11:03:17
2018-06-23 11:03:27
2018-06-23 11:03:37
2018-06-23 11:03:47
2018-06-23 11:03:57
2018-06-23 11:04:08
2018-06-23 11:04:18
2018-06-23 11:04:28
2018-06-23 11:04:38
2018-06-23 11:04:48
2018-06-23 11:04:58
2018-06-23 11:05:09
2018-06-23 11:05:19
2018-06-23 11:05:29
2018-06-23 11:05:39
2018-06-23 11:05:49
2018-06-23 11:05:59
2018-06-23 11:06:09
2018-06-23 11:06:19
2018-06-23 11:06:29
2018-06-23 11:06:39
2018-06-23 11:06:49
2018-06-23 11:07:00
2018-06-23 11:07:10
2018-06-23 11:07:20
2018-06-23 11:07:30
2018-06-23 11:07:40
2018-06-23 11:07:50
2018-06-23 11:08:00
2018-06-23 11:08:10
2018-06-23 11:08:20
2018-06-23 11:08:31
2018-06-23 11:08:41
2018-06-23 11:08:51
2018-06-23 11:09:01
2018-06-23 11:09:11
2018-06-23 11:09:21
2018-06-23 11:09:31
2018-06-23 11:09:41
2018-06-23 11:09:51
2018-06-23 11:10:02
2018-06-23 11:10:12
2018-06-23 11:10:22
2018-06-23 11:10:32


2018-06-23 12:19:24
2018-06-23 12:19:34
2018-06-23 12:19:44
2018-06-23 12:19:54
2018-06-23 12:20:04
2018-06-23 12:20:14
2018-06-23 12:20:24
2018-06-23 12:20:35
2018-06-23 12:20:45
2018-06-23 12:20:55
2018-06-23 12:21:05
2018-06-23 12:21:15
2018-06-23 12:21:25
2018-06-23 12:21:35
2018-06-23 12:21:45
2018-06-23 12:21:55
2018-06-23 12:22:05
2018-06-23 12:22:15
2018-06-23 12:22:25
2018-06-23 12:22:36
2018-06-23 12:22:46
2018-06-23 12:22:56
2018-06-23 12:23:06
2018-06-23 12:23:16
2018-06-23 12:23:26
2018-06-23 12:23:36
2018-06-23 12:23:46
2018-06-23 12:23:56
2018-06-23 12:24:06
2018-06-23 12:24:17
2018-06-23 12:24:27
2018-06-23 12:24:37
2018-06-23 12:24:47
2018-06-23 12:24:57
2018-06-23 12:25:07
2018-06-23 12:25:17
2018-06-23 12:25:27
2018-06-23 12:25:37
2018-06-23 12:25:47
2018-06-23 12:25:57
2018-06-23 12:26:07
2018-06-23 12:26:18
2018-06-23 12:26:28
2018-06-23 12:26:38
2018-06-23 12:26:48
2018-06-23 12:26:58
2018-06-23 12:27:09
2018-06-23 12:27:19
2018-06-23 12:27:29
2018-06-23 12:27:39


2018-06-23 13:49:05
2018-06-23 13:49:15
2018-06-23 13:49:25
2018-06-23 13:49:35
2018-06-23 13:49:45
2018-06-23 13:49:55
2018-06-23 13:50:05
2018-06-23 13:50:15
2018-06-23 13:50:26
2018-06-23 13:50:36
2018-06-23 13:50:46
2018-06-23 13:50:56
2018-06-23 13:51:06
2018-06-23 13:51:16
2018-06-23 13:51:26
2018-06-23 13:51:37
2018-06-23 13:51:47
2018-06-23 13:51:57
2018-06-23 13:52:07
2018-06-23 13:52:17
2018-06-23 13:52:27
2018-06-23 13:52:37
2018-06-23 13:52:47
2018-06-23 13:52:58
2018-06-23 13:53:08
2018-06-23 13:53:18
2018-06-23 13:53:28
2018-06-23 13:53:38
2018-06-23 13:53:48
2018-06-23 13:53:58
2018-06-23 13:54:09
2018-06-23 13:54:19
2018-06-23 13:54:29
2018-06-23 13:54:39
2018-06-23 13:54:49
2018-06-23 13:54:59
2018-06-23 13:55:09
2018-06-23 13:55:19
2018-06-23 13:55:29
2018-06-23 13:55:39
2018-06-23 13:55:49
2018-06-23 13:55:59
2018-06-23 13:56:10
2018-06-23 13:56:20
2018-06-23 13:56:30
2018-06-23 13:56:40
2018-06-23 13:56:50
2018-06-23 13:57:00
2018-06-23 13:57:10
2018-06-23 13:57:20


In [13]:
# Filter to only relevant lines
out_df = out_df[out_df.Line.isin(use_lines)]
print out_df.count()
out_df.head(10)

DateTime       1181952
TrainID        1181952
Line           1181952
Direction      1181952
StationName    1181952
dtype: int64


,DateTime,TrainID,Line,Direction,StationName
0,2018-06-10 13:59:00,181,BL,2.0,Farragut West
1,2018-06-10 13:59:00,288,BL,2.0,Stadium-Armory
6,2018-06-10 13:59:00,157,OR,2.0,Court House
19,2018-06-10 13:59:00,100,SV,2.0,Eastern Market
20,2018-06-10 13:59:00,160,SV,2.0,Foggy Bottom-GWU
23,2018-06-10 13:59:10,288,BL,2.0,Stadium-Armory
27,2018-06-10 13:59:10,130,OR,1.0,Virginia Square-GMU
39,2018-06-10 13:59:10,104,SV,1.0,Addison Road-Seat Pleasant
40,2018-06-10 13:59:10,160,SV,2.0,Foggy Bottom-GWU
43,2018-06-10 13:59:20,091,BL,1.0,Rosslyn


In [14]:
# Necessary for grouping by train, station, direction, since trains sit at a station for a while
out_df['id_station_direction'] = out_df.apply(lambda row: str(row['TrainID']) + row['StationName'] + str(row['Direction']), axis=1)
out_df.sort_values(['TrainID', 'DateTime'], inplace=True)
out_df.head(5)

,DateTime,TrainID,Line,Direction,StationName,id_station_direction
22589,2018-06-15 21:11:40,001,BL,2.0,Braddock Road,001Braddock Road2.0
22619,2018-06-15 21:11:50,001,BL,2.0,Braddock Road,001Braddock Road2.0
22644,2018-06-15 21:12:00,001,BL,2.0,Braddock Road,001Braddock Road2.0
22667,2018-06-15 21:12:10,001,BL,2.0,Braddock Road,001Braddock Road2.0
22776,2018-06-15 21:13:01,001,BL,2.0,King St-Old Town,001King St-Old Town2.0


In [15]:
adj_check = (out_df.id_station_direction != out_df.id_station_direction.shift()).cumsum()
grouped = out_df.groupby(['TrainID', 'StationName', 'Direction', adj_check], as_index=False, sort=False)['DateTime'].max()

In [16]:
print len(grouped)
grouped.head(10)

194133


,TrainID,StationName,Direction,DateTime
0,001,Braddock Road,2.0,2018-06-15 21:12:10
1,001,King St-Old Town,2.0,2018-06-15 21:14:02
2,001,Van Dorn Street,2.0,2018-06-15 21:19:35
3,001,Franconia-Springfield,2.0,2018-06-15 21:23:58
4,001,Franconia-Springfield,1.0,2018-06-15 21:44:31
5,001,Franconia-Springfield,2.0,2018-06-15 21:45:42
6,001,Van Dorn Street,1.0,2018-06-15 21:51:06
7,001,King St-Old Town,1.0,2018-06-15 21:56:10
8,001,Braddock Road,1.0,2018-06-15 21:58:11
9,001,Ronald Reagan Washington National Airport,1.0,2018-06-15 22:03:14


In [17]:
grouped['Date'] = grouped['DateTime'].map(lambda x: x.split(' ')[0])
grouped['Time'] = grouped['DateTime'].map(lambda x: x.split(' ')[1])
grouped.head(5)

,TrainID,StationName,Direction,DateTime,Date,Time
0,001,Braddock Road,2.0,2018-06-15 21:12:10,2018-06-15,21:12:10
1,001,King St-Old Town,2.0,2018-06-15 21:14:02,2018-06-15,21:14:02
2,001,Van Dorn Street,2.0,2018-06-15 21:19:35,2018-06-15,21:19:35
3,001,Franconia-Springfield,2.0,2018-06-15 21:23:58,2018-06-15,21:23:58
4,001,Franconia-Springfield,1.0,2018-06-15 21:44:31,2018-06-15,21:44:31


In [18]:
# Keep only the rows at the two relevant stations
grouped = grouped[grouped.StationName.isin([departure, arrival])]
grouped = grouped[grouped.apply(lambda x: time_relevant(x.Time), axis=1)]
len(grouped)

2397

In [19]:
# Sort
grouped.sort_values(['TrainID', 'DateTime'], inplace=True)
grouped.head(10)

,TrainID,StationName,Direction,DateTime,Date,Time
123,001,L'Enfant Plaza,1.0,2018-06-18 15:08:33,2018-06-18,15:08:33
144,001,L'Enfant Plaza,2.0,2018-06-18 16:06:05,2018-06-18,16:06:05
150,001,Foggy Bottom-GWU,2.0,2018-06-18 16:16:01,2018-06-18,16:16:01
168,001,Foggy Bottom-GWU,1.0,2018-06-18 17:18:35,2018-06-18,17:18:35
221,002,L'Enfant Plaza,2.0,2018-06-20 15:46:48,2018-06-20,15:46:48
227,002,Foggy Bottom-GWU,2.0,2018-06-20 15:57:45,2018-06-20,15:57:45
246,002,Foggy Bottom-GWU,1.0,2018-06-20 16:59:00,2018-06-20,16:59:00
252,002,L'Enfant Plaza,1.0,2018-06-20 17:10:40,2018-06-20,17:10:40
384,003,Foggy Bottom-GWU,1.0,2018-06-20 16:18:51,2018-06-20,16:18:51
390,003,L'Enfant Plaza,1.0,2018-06-20 16:28:59,2018-06-20,16:28:59


In [34]:
# Go through and determine which trains are on time and which aren't, and do some transformations of datetimes

header = ['TrainID', 'DepartureStation', 'ArrivalStation', 'DepartureTime', 'ArrivalTime', 'TravelTime', 'OnTime']
outputs = []
old_station = old_datetime = old_date = old_train = None
for rownum, row in grouped.iterrows():
    print "old: {0}, {1}, new: {2}, {3}".format(old_station, old_train, row['StationName'], row['TrainID'])
    new_datetime = datetime.datetime.strptime(row['DateTime'], '%Y-%m-%d %H:%M:%S')
    if old_station == departure and row['StationName'] == arrival and old_train == row['TrainID'] and old_date == row['Date']:
        print new_datetime
        on_time = 1 if is_on_time(new_datetime.time()) is True else 0
        out_row = [row['TrainID'], departure, arrival, old_datetime, new_datetime, new_datetime - old_datetime, on_time]
        outputs.append(out_row)
    old_station = row['StationName']
    old_train = row['TrainID']
    old_datetime = new_datetime
    old_date = row['Date']
newdf = pd.DataFrame(outputs, columns=header)

old: None, None, new: L'Enfant Plaza, 001
old: L'Enfant Plaza, 001, new: L'Enfant Plaza, 001
old: L'Enfant Plaza, 001, new: Foggy Bottom-GWU, 001
2018-06-18 16:16:01
old: Foggy Bottom-GWU, 001, new: Foggy Bottom-GWU, 001
old: Foggy Bottom-GWU, 001, new: L'Enfant Plaza, 002
old: L'Enfant Plaza, 002, new: Foggy Bottom-GWU, 002
2018-06-20 15:57:45
old: Foggy Bottom-GWU, 002, new: Foggy Bottom-GWU, 002
old: Foggy Bottom-GWU, 002, new: L'Enfant Plaza, 002
old: L'Enfant Plaza, 002, new: Foggy Bottom-GWU, 003
old: Foggy Bottom-GWU, 003, new: L'Enfant Plaza, 003
old: L'Enfant Plaza, 003, new: L'Enfant Plaza, 003
old: L'Enfant Plaza, 003, new: Foggy Bottom-GWU, 004
old: Foggy Bottom-GWU, 004, new: L'Enfant Plaza, 004
old: L'Enfant Plaza, 004, new: L'Enfant Plaza, 004
old: L'Enfant Plaza, 004, new: Foggy Bottom-GWU, 004
2018-06-19 16:34:08
old: Foggy Bottom-GWU, 004, new: L'Enfant Plaza, 006
old: L'Enfant Plaza, 006, new: Foggy Bottom-GWU, 006
2018-06-18 15:52:15
old: Foggy Bottom-GWU, 006, new:

old: Foggy Bottom-GWU, 143, new: L'Enfant Plaza, 143
old: L'Enfant Plaza, 143, new: L'Enfant Plaza, 143
old: L'Enfant Plaza, 143, new: Foggy Bottom-GWU, 143
2018-06-18 17:13:42
old: Foggy Bottom-GWU, 143, new: L'Enfant Plaza, 145
old: L'Enfant Plaza, 145, new: Foggy Bottom-GWU, 145
2018-06-10 15:53:34
old: Foggy Bottom-GWU, 145, new: Foggy Bottom-GWU, 145
old: Foggy Bottom-GWU, 145, new: L'Enfant Plaza, 145
old: L'Enfant Plaza, 145, new: Foggy Bottom-GWU, 146
old: Foggy Bottom-GWU, 146, new: Foggy Bottom-GWU, 146
old: Foggy Bottom-GWU, 146, new: L'Enfant Plaza, 146
old: L'Enfant Plaza, 146, new: L'Enfant Plaza, 146
old: L'Enfant Plaza, 146, new: Foggy Bottom-GWU, 146
2018-06-10 17:23:23
old: Foggy Bottom-GWU, 146, new: L'Enfant Plaza, 146
old: L'Enfant Plaza, 146, new: Foggy Bottom-GWU, 146
2018-06-14 16:27:24
old: Foggy Bottom-GWU, 146, new: Foggy Bottom-GWU, 147
old: Foggy Bottom-GWU, 147, new: L'Enfant Plaza, 147
old: L'Enfant Plaza, 147, new: L'Enfant Plaza, 147
old: L'Enfant Plaza

old: L'Enfant Plaza, 256, new: Foggy Bottom-GWU, 256
2018-06-16 16:45:47
old: Foggy Bottom-GWU, 256, new: Foggy Bottom-GWU, 256
old: Foggy Bottom-GWU, 256, new: Foggy Bottom-GWU, 256
old: Foggy Bottom-GWU, 256, new: L'Enfant Plaza, 256
old: L'Enfant Plaza, 256, new: Foggy Bottom-GWU, 258
old: Foggy Bottom-GWU, 258, new: L'Enfant Plaza, 258
old: L'Enfant Plaza, 258, new: L'Enfant Plaza, 258
old: L'Enfant Plaza, 258, new: Foggy Bottom-GWU, 258
2018-06-14 16:43:20
old: Foggy Bottom-GWU, 258, new: Foggy Bottom-GWU, 258
old: Foggy Bottom-GWU, 258, new: L'Enfant Plaza, 258
old: L'Enfant Plaza, 258, new: L'Enfant Plaza, 259
old: L'Enfant Plaza, 259, new: Foggy Bottom-GWU, 259
2018-06-12 15:34:31
old: Foggy Bottom-GWU, 259, new: Foggy Bottom-GWU, 259
old: Foggy Bottom-GWU, 259, new: L'Enfant Plaza, 259
old: L'Enfant Plaza, 259, new: Foggy Bottom-GWU, 260
old: Foggy Bottom-GWU, 260, new: L'Enfant Plaza, 260
old: L'Enfant Plaza, 260, new: L'Enfant Plaza, 260
old: L'Enfant Plaza, 260, new: Foggy 

old: Foggy Bottom-GWU, 369, new: Foggy Bottom-GWU, 371
old: Foggy Bottom-GWU, 371, new: L'Enfant Plaza, 371
old: L'Enfant Plaza, 371, new: L'Enfant Plaza, 371
old: L'Enfant Plaza, 371, new: Foggy Bottom-GWU, 371
2018-06-11 16:32:42
old: Foggy Bottom-GWU, 371, new: L'Enfant Plaza, 371
old: L'Enfant Plaza, 371, new: Foggy Bottom-GWU, 371
2018-06-12 15:55:34
old: Foggy Bottom-GWU, 371, new: Foggy Bottom-GWU, 371
old: Foggy Bottom-GWU, 371, new: L'Enfant Plaza, 371
old: L'Enfant Plaza, 371, new: Foggy Bottom-GWU, 373
old: Foggy Bottom-GWU, 373, new: L'Enfant Plaza, 373
old: L'Enfant Plaza, 373, new: L'Enfant Plaza, 373
old: L'Enfant Plaza, 373, new: Foggy Bottom-GWU, 373
2018-06-11 17:28:16
old: Foggy Bottom-GWU, 373, new: Foggy Bottom-GWU, 373
old: Foggy Bottom-GWU, 373, new: L'Enfant Plaza, 373
old: L'Enfant Plaza, 373, new: L'Enfant Plaza, 373
old: L'Enfant Plaza, 373, new: Foggy Bottom-GWU, 373
2018-06-12 17:13:32
old: Foggy Bottom-GWU, 373, new: Foggy Bottom-GWU, 373
old: Foggy Bottom

old: L'Enfant Plaza, 492, new: L'Enfant Plaza, 492
old: L'Enfant Plaza, 492, new: Foggy Bottom-GWU, 492
2018-06-20 15:34:40
old: Foggy Bottom-GWU, 492, new: Foggy Bottom-GWU, 492
old: Foggy Bottom-GWU, 492, new: L'Enfant Plaza, 492
old: L'Enfant Plaza, 492, new: L'Enfant Plaza, 493
old: L'Enfant Plaza, 493, new: L'Enfant Plaza, 493
old: L'Enfant Plaza, 493, new: L'Enfant Plaza, 493
old: L'Enfant Plaza, 493, new: Foggy Bottom-GWU, 493
2018-06-22 16:21:17
old: Foggy Bottom-GWU, 493, new: Foggy Bottom-GWU, 494
old: Foggy Bottom-GWU, 494, new: Foggy Bottom-GWU, 494
old: Foggy Bottom-GWU, 494, new: L'Enfant Plaza, 494
old: L'Enfant Plaza, 494, new: L'Enfant Plaza, 494
old: L'Enfant Plaza, 494, new: L'Enfant Plaza, 494
old: L'Enfant Plaza, 494, new: Foggy Bottom-GWU, 494
2018-06-20 16:08:14
old: Foggy Bottom-GWU, 494, new: Foggy Bottom-GWU, 494
old: Foggy Bottom-GWU, 494, new: Foggy Bottom-GWU, 495
old: Foggy Bottom-GWU, 495, new: L'Enfant Plaza, 495
old: L'Enfant Plaza, 495, new: L'Enfant P

In [35]:
newdf.head(10)

,TrainID,DepartureStation,ArrivalStation,DepartureTime,ArrivalTime,TravelTime,OnTime
0,001,L'Enfant Plaza,Foggy Bottom-GWU,2018-06-18 16:06:05,2018-06-18 16:16:01,00:09:56,0
1,002,L'Enfant Plaza,Foggy Bottom-GWU,2018-06-20 15:46:48,2018-06-20 15:57:45,00:10:57,0
2,004,L'Enfant Plaza,Foggy Bottom-GWU,2018-06-19 16:21:19,2018-06-19 16:34:08,00:12:49,0
3,006,L'Enfant Plaza,Foggy Bottom-GWU,2018-06-18 15:40:16,2018-06-18 15:52:15,00:11:59,0
4,007,L'Enfant Plaza,Foggy Bottom-GWU,2018-06-19 17:11:27,2018-06-19 17:22:15,00:10:48,0
5,008,L'Enfant Plaza,Foggy Bottom-GWU,2018-06-11 16:27:03,2018-06-11 16:37:47,00:10:44,0
6,008,L'Enfant Plaza,Foggy Bottom-GWU,2018-06-13 15:06:38,2018-06-13 15:24:13,00:17:35,0
7,008,L'Enfant Plaza,Foggy Bottom-GWU,2018-06-14 15:36:35,2018-06-14 15:48:58,00:12:23,0
8,008,L'Enfant Plaza,Foggy Bottom-GWU,2018-06-18 15:14:47,2018-06-18 15:25:25,00:10:38,0
9,008,L'Enfant Plaza,Foggy Bottom-GWU,2018-06-21 15:35:53,2018-06-21 15:46:41,00:10:48,0


In [37]:
newdf.dropna(inplace=True)
newdf['justtime'] = newdf['DepartureTime'].map(lambda x: x.time())
newdf['timefrommin'] = newdf['justtime'].map(lambda x: (datetime.datetime.combine(date.today(), x) - datetime.datetime.combine(date.today(), min(newdf['justtime'])) ).seconds)
newdf.head(10)

,TrainID,DepartureStation,ArrivalStation,DepartureTime,ArrivalTime,TravelTime,OnTime,justtime,timefrommin
0,001,L'Enfant Plaza,Foggy Bottom-GWU,2018-06-18 16:06:05,2018-06-18 16:16:01,00:09:56,0,16:06:05,3954
1,002,L'Enfant Plaza,Foggy Bottom-GWU,2018-06-20 15:46:48,2018-06-20 15:57:45,00:10:57,0,15:46:48,2797
2,004,L'Enfant Plaza,Foggy Bottom-GWU,2018-06-19 16:21:19,2018-06-19 16:34:08,00:12:49,0,16:21:19,4868
3,006,L'Enfant Plaza,Foggy Bottom-GWU,2018-06-18 15:40:16,2018-06-18 15:52:15,00:11:59,0,15:40:16,2405
4,007,L'Enfant Plaza,Foggy Bottom-GWU,2018-06-19 17:11:27,2018-06-19 17:22:15,00:10:48,0,17:11:27,7876
5,008,L'Enfant Plaza,Foggy Bottom-GWU,2018-06-11 16:27:03,2018-06-11 16:37:47,00:10:44,0,16:27:03,5212
6,008,L'Enfant Plaza,Foggy Bottom-GWU,2018-06-13 15:06:38,2018-06-13 15:24:13,00:17:35,0,15:06:38,387
7,008,L'Enfant Plaza,Foggy Bottom-GWU,2018-06-14 15:36:35,2018-06-14 15:48:58,00:12:23,0,15:36:35,2184
8,008,L'Enfant Plaza,Foggy Bottom-GWU,2018-06-18 15:14:47,2018-06-18 15:25:25,00:10:38,0,15:14:47,876
9,008,L'Enfant Plaza,Foggy Bottom-GWU,2018-06-21 15:35:53,2018-06-21 15:46:41,00:10:48,0,15:35:53,2142


In [38]:
# Bin some times to get intervals, both named and numbered
import math
newdf['binned_mins_since_start'] = newdf['timefrommin'].map(lambda x: bin_seconds * math.floor(x / bin_seconds) / 60)
newdf['binned_mins_str'] = newdf['binned_mins_since_start'].map(lambda x: (datetime.datetime.strptime(min_time, '%H:%M:%S') + timedelta(minutes=x)).strftime('%I:%M %p'))
newdf = newdf[['binned_mins_str', 'binned_mins_since_start', 'OnTime']]
newdf.head(10)

,binned_mins_str,binned_mins_since_start,OnTime
0,04:03 PM,63.0,0
1,03:45 PM,45.0,0
2,04:21 PM,81.0,0
3,03:39 PM,39.0,0
4,05:09 PM,129.0,0
5,04:24 PM,84.0,0
6,03:06 PM,6.0,0
7,03:36 PM,36.0,0
8,03:12 PM,12.0,0
9,03:33 PM,33.0,0


In [45]:
# Save to a .csv
print len(newdf)
newdf.to_csv(outcomes_file, index=False)

538


In [40]:
newdf.groupby('binned_mins_str').agg({'OnTime': ('sum', 'count')})

OnTime      
                   sum count
binned_mins_str             
03:00 PM             0    13
03:03 PM             0    11
03:06 PM             0    11
03:09 PM             0     9
03:12 PM             0    12
03:15 PM             0     9
03:18 PM             0    14
03:21 PM             0    11
03:24 PM             0    11
03:27 PM             0    11
03:30 PM             0    14
03:33 PM             0    10
03:36 PM             0    11
03:39 PM             0    15
03:42 PM             0    11
03:45 PM             0    13
03:48 PM             0    15
03:51 PM             0    14
03:54 PM             0     8
03:57 PM             0    12
04:00 PM             0    12
04:03 PM             0    12
04:06 PM             0    12
04:09 PM             0    13
04:12 PM             0    11
04:15 PM             0    12
04:18 PM             0    10
04:21 PM             0    14
04:24 PM             0    12
04:27 PM             0    12
04:30 PM             0    12
04:33 PM             0    14
04:36 PM             1    10
04:39 PM            11    12
04:42 PM            11    11
04:45 PM            10    10
04:48 PM             4    11
04:51 PM             0    12
04:54 PM             0    11
04:57 PM             0    11
05:00 PM             0    13
05:03 PM             0    10
05:06 PM             0    13
05:09 PM             0     8
05:12 PM             0    12
05:15 PM             0    11
05:18 PM             0     2